# Constants

In [ ]:
import os

CHART_TIME_STRING = "12h, 15m, 1d, 1h, 1m, 1mo, 1w, 2h, 30m, 3d, 3m, 4h, 5m, 6h, 8h"
BINANCE_MONTHLY_URL = "https://data.binance.vision/data/futures/um/monthly/klines/"
BINANCE_DAILY_URL = "https://data.binance.vision/data/futures/um/daily/klines/"

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
download_dir = os.path.join(parent_dir, "data", "downloaded_data")
output_dir = os.path.join(parent_dir, "data", "extracted_data")
print(download_dir)
print(output_dir)

# Creating Arrays

In [ ]:
CHART_TIME_STRING = CHART_TIME_STRING.replace(" ", "")
CHART_TIME_ARRAY = CHART_TIME_STRING.split(",")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta

# Setup Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Correct the ChromeDriver installation path issue
chrome_install = ChromeDriverManager().install()

# Extract the folder and find the chromedriver.exe
folder = os.path.dirname(chrome_install)
chromedriver_path = os.path.join(folder, "chromedriver.exe")

# Create the Service object using the corrected chromedriver path
service = Service(chromedriver_path)

# Create a new instance of the Chrome driver with the service object and options
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the website
driver.get("https://data.binance.vision/?prefix=data/futures/um/monthly/klines/")

symbols = []
trend = []

# Wait for the JavaScript to load (will not display UI)
time.sleep(5)

# Retrieve data from the current page
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# Parse the HTML content using BeautifulSoup
containers = soup.find_all('div', class_='container-md')[1]
table_rows = containers.find('tbody', {'id': 'listing'}).find_all("tr")
URL_ARRAY = [row.find("td").find('a').get('href') for row in table_rows]
URL_ARRAY.pop(0)
URL_ARRAY = [x.replace('?prefix=', '') for x in URL_ARRAY]

# Set the start date and end date
start_date = datetime(2020, 1, 1)
end_date = datetime(2080, 1, 1)

# Initialize the list of year-month pairs
MONTH_ARRAY = []

# Iterate over the dates from start to end
date = start_date
while date < end_date:
    # Format the date as a year-month pair
    year_month = date.strftime("%Y-%m")
    #
    # Add the year-month pair to the list
    MONTH_ARRAY.append(year_month)

    # Increment the date by one month
    date += timedelta(days=31)

SYMBOL_ARRAY = []
for url in URL_ARRAY:
    symbol = url.replace("https://data.binance.vision/data/futures/um/monthly/klines/", "").replace("/", "")
    SYMBOL_ARRAY.append(symbol)

print(SYMBOL_ARRAY)
print(MONTH_ARRAY)
print(CHART_TIME_ARRAY)
print("Symbol, month, and chart arrays are successfully created")

# Quit the driver
driver.quit()
